# Preperation

In [1]:
import numpy as np
import pandas as pd
import optuna
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, f1_score
import matplotlib.pyplot as plt
import datetime as dt

Importing plotly failed. Interactive plots will not work.


In [2]:
import formatting
import optimization
import visualization
import valuation

In [4]:
df_er, df_dint, df_yint = formatting.read()
df_er, df_dint, df_yint = formatting.format(df_er, df_dint, df_yint)

# US interest rate prediction

In [11]:
df_dint_p = formatting.format_prophet(df_dint, date_start='1993-10-1', date_end='2023-09-01')

In [9]:
test_size = 36

In [10]:
df_dint_train, df_dint_test = optimization.split_train_test(df_dint_p, test_size)

In [8]:
# visualization.visualize(df_dint_train, df_dint_test)

In [ ]:
study_d = optimization.get_best_params(df_dint_train, df_dint_test, test_size)

In [ ]:
# train the model and see the accuracy
m_d = Prophet(**study_d.best_params)
m_d.fit(df_dint_train)
visualization.assess(m_d, test_size, df_dint_p, df_dint_train, df_dint_test)

In [ ]:
# predict future dollar interest rate
m_d = Prophet(**study_d.best_params)
m_d.fit(df_dint_p)
future_d = m_d.make_future_dataframe(periods=12, freq='M')
forecast_d = m_d.predict(future_d)

In [ ]:
plot.plot_plotly(m_d, forecast_d)

# Yen interest rate prediction

In [12]:
df_yint_p = formatting.format_prophet(df_yint, date_start='1993-10-1', date_end='2023-09-01')

In [13]:
test_size = 36

In [14]:
df_yint_train, df_yint_test = optimization.split_train_test(df_yint_p, test_size)

In [ ]:
# visualization.visualize(df_dint_train, df_dint_test)

In [ ]:
study_y = optimization.get_best_params(df_yint_train, df_yint_test, test_size)

In [ ]:
# train the model and see the accuracy
m_y = Prophet(**study_y.best_params)
m_y.fit(df_yint_train)
visualization.assess(m_y, test_size, df_yint_p, df_yint_train, df_yint_test)

In [ ]:
# predict future dollar interest rate
m_y = Prophet(**study_y.best_params)
m_y.fit(df_yint_p)
future_y = m_y.make_future_dataframe(periods=12, freq='M')
forecast_y = m_y.predict(future_y)

In [ ]:
plot.plot_plotly(m_y, forecast_y)

# Exchange rate prediction

In [15]:
df_er_p = formatting.format_prophet(df_er, date_start='1993-10-1', date_end='2023-09-01')

In [18]:
df = formatting.format_er(df_er_p, df_dint_p, df_yint_p)

In [19]:
test_size=36
df_train, df_test = optimization.split_train_test(df, test_size)

In [ ]:
study = optimization.get_best_params(df_train, df_test, test_size)

In [ ]:
# train the model
m = Prophet(**study.best_params)
m.fit(df_train)
# scores
visualization.assess(m, test_length, df, df_train, df_test)

In [ ]:
# create a prophet model
m = Prophet(**study.best_params)

m.add_regressor('ir_dollar')
m.add_regressor('ir_yen')
m.fit(df)

In [ ]:
# get a dataframe for prediciton and predict
# cap is the highest limit price set for prediction
# future_start_date is the first datetime of which the model should predict the exchange rate
# Because Prophet.make_future_dataframe() creates datetimes of the end day of each month,
# we set 2023-09-30
future = formatting.format_er_future(df_dint_p, forecast_d, df_yint_p, forecast_y, 
                                     cap=158, future_start_date='2023-09-30')
forecast = m.predict(future)

In [ ]:
plot.plot_plotly(m, forecast)

In [ ]:
plot.plot_components_plotly(m, forecast)

# References